In [29]:
import pandas as pd 
import numpy as np

In [30]:
df= pd.read_csv(r'reservas 27032024 27032025.csv', sep=';', skiprows=1, encoding='utf-8')
df2= pd.read_csv(r'C:\Users\tabo_\OneDrive\Desktop\absolute\reporte de ingreso año 2024 a la fecha.csv', sep=';', encoding='latin1')
df3=pd.read_excel(r'C:\Users\tabo_\OneDrive\Desktop\absolute\Check-in del 2024-03-27 al 2025-03-28 bookin.xls')

In [31]:
df3

,Número de reserva,Reservado por,Nombre del cliente (o clientes),Entrada,Salida,Fecha de reserva,Estado,Habitaciones,Personas,Adultos,...,Comentarios,Grupo de reserva,Booker country,Motivo del viaje,Dispositivo,Tipo de unidad,Duración (noches),Fecha de cancelación,Dirección,Número de teléfono
0,4396111470,"Parra, John S.",John S. Parra,2024-03-27,2024-03-28,2024-03-27 12:11:31,ok,1,1,NaN,...,NaN,NaN,co,Ocio,Móvil,Dormitorio,1,NaN,NaN,NaN
1,4557704876,"López, Jojanna",Jojanna López,2024-03-27,2024-03-28,2024-03-25 10:32:33,cancelled_by_guest,1,1,NaN,...,NaN,NaN,co,Ocio,Móvil,Dormitorio,1,2024-03-26 19:32:31,NaN,NaN
2,4637253976,"Surós, Aldo",Aldo Surós,2024-03-27,2024-03-28,2024-03-27 18:40:32,ok,1,1,NaN,...,NaN,NaN,cr,NaN,Móvil,Dormitorio,1,NaN,NaN,NaN
3,4689813600,"Le Roy, Victor",Victor Le Roy,2024-03-27,2024-03-28,2024-03-27 21:06:36,ok,1,1,NaN,...,NaN,NaN,co,Ocio,Móvil,Dormitorio,1,NaN,NaN,NaN
4,4897263353,"Hernandez, Ana Milena",Ana Milena Hernandez,2024-03-27,2024-03-28,2024-03-27 20:18:03,ok,1,3,NaN,...,NaN,NaN,co,Ocio,Móvil,Basic Triple Room with Shared Bathroom,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2281,4502890135,"moreno, liseth",liseth moreno,2025-03-20,2025-03-23,2025-02-01 13:21:39,cancelled_by_guest,1,4,4.0,...,NaN,NaN,co,Ocio,Móvil,Quadruple Room with Shared Bathroom,3,2025-03-04 00:05:11,NaN,NaN
2282,4749774277,"Trejos Cueto, José Manuel",José Manuel Trejos Cueto,2025-03-22,2025-03-26,2025-01-22 08:00:41,ok,1,1,NaN,...,Approximate time of arrival: between 10:00 and...,NaN,co,Ocio,Móvil,Habitacion individual economica en lavanderia ...,4,NaN,NaN,NaN
2283,4942788539,"Magdalena, Koneczna",Koneczna Magdalena,2025-03-23,2025-03-27,2025-02-21 08:59:49,ok,1,1,NaN,...,Approximate time of arrival: between 21:00 and...,NaN,pl,Ocio,Ordenador,Dormitorio,4,NaN,NaN,NaN
2284,4517617109,"Larouche, Mario",Mario Larouche,2025-03-25,2025-03-27,2025-02-13 18:07:20,cancelled_by_guest,1,1,NaN,...,NaN,NaN,ca,Ocio,Móvil,Dormitorio,2,2025-03-11 01:02:56,NaN,NaN


In [32]:
#df.info()
df.isnull().sum()

Titular                               0
Tipo doc                             85
NÃºmero doc                         103
Nacionalidad                        102
Email                              2118
Fecha de nacimiento                 180
GÃ©nero                             328
HabitaciÃ³n                           0
Ingreso                               0
Salida                                0
OcupaciÃ³n(NÃºmero de personas)       0
Plan                               2467
Canal                              2238
Creada por                            0
Fecha creaciÃ³n                       0
CÃ³digo de la reserva                 0
Total alojamiento                     0
Estatus                            2065
ID CategorÃ­a                         0
Myallocator ID categorÃ­a          2467
Myallocator ID                     2467
Myallocator ID de agencia          2467
dtype: int64

In [33]:
# Eliminar las columnas especificadas
columns_to_drop = ['Email', 'Plan', 'Canal', 'Estatus', 
                   'Myallocator ID categoría', 'Myallocator ID', 'Myallocator ID de agencia']

# Asegúrate de que el DataFrame que estás procesando es 'df'
df = df.drop(columns=columns_to_drop, errors='ignore')

# Verificar las columnas restantes
print(df.columns)

Index(['Titular', 'Tipo doc', 'NÃºmero doc', 'Nacionalidad',
       'Fecha de nacimiento', 'GÃ©nero', 'HabitaciÃ³n', 'Ingreso', 'Salida',
       'OcupaciÃ³n(NÃºmero de personas)', 'Creada por', 'Fecha creaciÃ³n',
       'CÃ³digo de la reserva', 'Total alojamiento', 'ID CategorÃ­a',
       'Myallocator ID categorÃ­a'],
      dtype='object')


In [34]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10052 entries, 0 to 10051
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Fecha           10052 non-null  object 
 1   Cliente         8414 non-null   object 
 2   Tipo doc        8152 non-null   object 
 3   Producto        10052 non-null  object 
 4   Descripción     766 non-null    object 
 5   Valor           10052 non-null  object 
 6   Forma de pago   10052 non-null  object 
 7   Registrado por  10052 non-null  object 
 8   Reserva #       6722 non-null   float64
 9   Factura #       48 non-null     object 
 10  Transacción #   10052 non-null  int64  
 11  Comprobante #   3 non-null      float64
dtypes: float64(2), int64(1), object(9)
memory usage: 942.5+ KB


In [35]:
# Eliminar las columnas 'Factura #' y 'Comprobante #'
columns_to_drop = ['Factura #', 'Comprobante #','Descripción']

# Asegúrate de que el DataFrame que estás procesando es 'df'
df = df.drop(columns=columns_to_drop, errors='ignore')

# Verificar las columnas restantes
print(df.columns)

Index(['Titular', 'Tipo doc', 'NÃºmero doc', 'Nacionalidad',
       'Fecha de nacimiento', 'GÃ©nero', 'HabitaciÃ³n', 'Ingreso', 'Salida',
       'OcupaciÃ³n(NÃºmero de personas)', 'Creada por', 'Fecha creaciÃ³n',
       'CÃ³digo de la reserva', 'Total alojamiento', 'ID CategorÃ­a',
       'Myallocator ID categorÃ­a'],
      dtype='object')


In [38]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2286 entries, 0 to 2285
Data columns (total 27 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Número de reserva                2286 non-null   int64  
 1   Reservado por                    2286 non-null   object 
 2   Nombre del cliente (o clientes)  2257 non-null   object 
 3   Entrada                          2286 non-null   object 
 4   Salida                           2286 non-null   object 
 5   Fecha de reserva                 2286 non-null   object 
 6   Estado                           2286 non-null   object 
 7   Habitaciones                     2286 non-null   int64  
 8   Personas                         2286 non-null   int64  
 9   Adultos                          28 non-null     float64
 10  Niños                            28 non-null     float64
 11  Edades de los niños:             1 non-null      float64
 12  Precio              

In [37]:
df3.isnull().sum()

Número de reserva                     0
Reservado por                         0
Nombre del cliente (o clientes)      29
Entrada                               0
Salida                                0
Fecha de reserva                      0
Estado                                0
Habitaciones                          0
Personas                              0
Adultos                            2258
Niños                              2258
Edades de los niños:               2285
Precio                                0
Comisión %                            0
Importe de la comisión              890
Estado del pago                    2284
Forma de pago                      2286
Comentarios                        1717
Grupo de reserva                   2286
Booker country                        0
Motivo del viaje                    504
Dispositivo                           6
Tipo de unidad                        0
Duración (noches)                     0
Fecha de cancelación               1730
